In [1]:
import PySpice.Logging.Logging as Logging
from PySpice.Spice.Netlist import Circuit
from PySpice.Unit import *

logger = Logging.setup_logging()

# -------------------------------------------------------------
# Define Circuit
# -------------------------------------------------------------
circuit = Circuit("Avalanche Pulse Generator")

# --- Power supplies ---
circuit.V('1', 'Vin', circuit.gnd, 1.5@u_V)           # AA cell (not used directly here)
circuit.V('bias', 'Vcc', circuit.gnd, 90@u_V)         # Boosted 90 V supply

# --- Collector resistor RC ---
circuit.R('C', 'Vcc', 'Cnode', 100@u_kOhm)            # RC between Vcc and transistor collector to be 100kohm

# --- Bias network for base --- basically using the voltage divider rule
circuit.R('6', 'Vcc', 'bias_node', 1@u_MOhm)          # divider upper resistor
circuit.R('2', 'bias_node', circuit.gnd, 10@u_MOhm)   # divider lower resistor (helps in creating a reference voltage without a second battery)
circuit.R('B', 'bias_node', 'Bnode', 10@u_kOhm)       # RB from divider to base to be 10kohm

# --- Transistor Q1 ---
# Node order: collector, base, emitter
circuit.BJT('Q1', 'Cnode', 'Bnode', circuit.gnd, model='2N2369') #2n2369 is good for avalanching

# --- Load resistor ---
circuit.R('L', 'Cnode', circuit.gnd, 50@u_Ohm)        # 50 Ω load connected at collector. this is where the spike is generated and hence the output.

# --- Storage capacitor (C_Ace) ---
circuit.C('Ace', 'Cnode', circuit.gnd, 2@u_pF)        # When the transistor suddenly breaks down (avalanches), it provides a path 
#for this stored energy to be dumped very quickly through the load resistor, creating the powerful, fast pulse. 
#This component is shown in the right-hand equivalent circuit.

# Insert 2N2369 model 
circuit.raw_spice += """
.model 2N2369 NPN (
+ IS=14.34f BF=255.9 NF=1.036 VAF=74.03 IKF=63.21m
+ ISE=14.34f NE=1.5 BR=6.092 NR=1.004 VAR=10.73 IKR=10m
+ ISC=0 NC=2 RB=10 IRB=0.1m RBM=5 RE=0.1 RC=0.3
+ CJE=26.19p VJE=0.75 MJE=0.33 TF=0.53n
+ XTF=3 VTF=4 ITF=1
+ CJC=9.52p VJC=0.55 MJC=0.33 XCJC=0.9
+ TR=50n CJS=0 VJS=0.75 MJS=0.5
+ EG=1.11 XTB=1.5 KF=0 AF=1 )
"""

# Simulation
simulator = circuit.simulator(temperature=25, nominal_temperature=25) #sim temp set at 25
analysis = simulator.transient(step_time=2@u_ps, end_time=50@u_ns)


2025-09-22 11:52:06,222 - PySpice.Spice.NgSpice.Shared.NgSpiceShared - Shared.WARNING - Warning: can't find the initialization file spinit.
2025-09-22 11:52:06,237 - PySpice.Spice.NgSpice.Shared.NgSpiceShared - Shared.WARNING - Unsupported Ngspice version 44
